# Beer dataset


## Positive dataset

1) Loading the data

In [1]:
import pandas as pd
df = pd.read_csv('ExampleDatasets/Beer/Beer_pos.csv')
df.head()

,ids,rel.ids,mzs,RTs,Int
0,1,0,116.070544,45.770423,2.170017e+09
1,88,0,117.073678,45.787586,1.256520e+08
2,372,0,70.065196,45.795901,3.487656e+07
3,501,0,231.133673,46.183948,2.519223e+07
4,1885,0,191.102328,44.427725,6.799680e+06


2) Loading the database

In [9]:
from ipaPy2 import ipa
DB=pd.read_csv('DB/IPA_MS1.csv')
adducts = pd.read_csv('DB/adducts.csv')
Bio = pd.read_csv('DB/allBIO_reactions.csv')

3) updating the database based on knowledge previously gained

In [10]:
updates = pd.read_csv('ExampleDatasets/Beer/update_based_on_standards.csv')

In [11]:
for k in range(0,len(updates.index)):
    DB.iloc[DB['id']==updates.iloc[k,0],5] = updates.iloc[k,7]
    DB.iloc[DB['id']==updates.iloc[k,0],6] = updates.iloc[k,8]
    DB.iloc[DB['id']==updates.iloc[k,0],7] = updates.iloc[k,10]

4) Running the whole IPA pipeline

In [12]:
annotations = ipa.simpleIPA(df=df,ionisation=1,DB=DB,adductsAll=adducts,ppm=3,Bio=Bio,
                            delta_add=0.1,delta_bio=0.1,burn=10,noits=100,ncores=5)

mapping isotope patterns ....
4.2 seconds elapsed
computing all adducts - Parallelized ....
32.4 seconds elapsed
annotating based on MS1 information - Parallelized ...
30.9 seconds elapsed
computing posterior probabilities including biochemical and adducts connections
initialising sampler ...


Gibbs Sampler Progress Bar: 100%|█████████████| 100/100 [04:48<00:00,  2.89s/it]


parsing results ...
Done -  295.5 seconds elapsed


5) Finally, we can show the results for one feature as an example

In [25]:
annotations[1].sort_values(by=['post Gibbs'], ascending=False)

,id,name,formula,adduct,m/z,charge,RT range,ppm,isotope pattern score,fragmentation pattern score,prior,post,post Gibbs,chi-square pval
2,C16435,Proline,C5H10NO2,M+H,116.070605,1,None,-0.523629,0.199500,None,0.194407,0.190411,0.277778,0.025428
0,C00148,L-Proline,C5H10NO2,M+H,116.070605,1,20;70,-0.523629,0.199500,None,0.194407,0.238014,0.244444,0.025428
3,C18170,3-Acetamidopropanal,C5H10NO2,M+H,116.070605,1,None,-0.523629,0.199500,None,0.194407,0.190411,0.211111,0.025428
1,C00763,D-Proline,C5H10NO2,M+H,116.070605,1,None,-0.523629,0.199500,None,0.194407,0.190411,0.200000,0.025428
4,NPA018555,Pleurocybellaziridin,C5H10NO2,M+H,116.070605,1,None,-0.523629,0.199500,None,0.194407,0.190411,0.066667,0.025428
5,Unknown,Unknown,None,None,None,None,None,3.000000,0.002501,None,0.027963,0.000343,0.000000,0.025428


## Negative dataset
1) Loading negative dataset

In [14]:
dfneg = pd.read_csv('ExampleDatasets/Beer/Beer_neg.csv')
dfneg.head()

,ids,rel.ids,mzs,RTs,Int
0,1,0,96.958464,48.370133,1.449591e+09
1,35,0,98.954139,48.395777,6.921542e+07
2,215,0,705.192297,50.280204,1.527424e+07
3,250,0,925.235146,49.086028,1.360392e+07
4,301,0,786.218066,50.939859,1.154266e+07


2) Running the whole IPA pipeline

In [26]:
annotationsNeg = ipa.simpleIPA(df=dfneg,ionisation=-1,DB=DB,adductsAll=adducts,ppm=10,Bio=Bio,
                            delta_add=0.1,delta_bio=0.1,burn=10,noits=100,ncores=5)

mapping isotope patterns ....
8.1 seconds elapsed
computing all adducts - Parallelized ....
135.5 seconds elapsed
annotating based on MS1 information - Parallelized ...
73.6 seconds elapsed
computing posterior probabilities including biochemical and adducts connections
initialising sampler ...


Gibbs Sampler Progress Bar: 100%|█████████████| 100/100 [34:39<00:00, 20.79s/it]


parsing results ...
Done -  2093.3 seconds elapsed


3) Finally, we can show the results for one feature as an example

In [27]:
annotations[1]

,id,name,formula,adduct,m/z,charge,RT range,ppm,isotope pattern score,fragmentation pattern score,prior,post,post Gibbs,chi-square pval
0,C00148,L-Proline,C5H10NO2,M+H,116.070605,1,20;70,-0.523629,0.199500,None,0.194407,0.238014,0.244444,0.025428
1,C00763,D-Proline,C5H10NO2,M+H,116.070605,1,None,-0.523629,0.199500,None,0.194407,0.190411,0.200000,0.025428
2,C16435,Proline,C5H10NO2,M+H,116.070605,1,None,-0.523629,0.199500,None,0.194407,0.190411,0.277778,0.025428
3,C18170,3-Acetamidopropanal,C5H10NO2,M+H,116.070605,1,None,-0.523629,0.199500,None,0.194407,0.190411,0.211111,0.025428
4,NPA018555,Pleurocybellaziridin,C5H10NO2,M+H,116.070605,1,None,-0.523629,0.199500,None,0.194407,0.190411,0.066667,0.025428
5,Unknown,Unknown,None,None,None,None,None,3.000000,0.002501,None,0.027963,0.000343,0.000000,0.025428
